In [ ]:
from keras.activations import leaky_relu
from sklearn.model_selection import train_test_split as tts
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.regularizers import l1_l2, l2, l1
from tensorflow.keras.optimizers.schedules import ExponentialDecay
# for images, use Conv2D, for regression, use Conv1Dr
from tensorflow.keras.layers import Input, BatchNormalization, Dense, Activation, Dropout, LeakyReLU

In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('drive/MyDrive/Colab Notebooks/data/telecom_churn.csv')

In [ ]:
X = data.loc[:, 'AccountWeeks':].values
#y = data['Churn'].values
y = to_categorical(data['Churn'].values,2)

In [ ]:
lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=1e-3/200)
optimizer = Adam(learning_rate=lr_schedule)

model = Sequential()
# model.add(Dense(16,activation=LeakyReLU(alpha=0.08),use_bias=True,kernel_regularizer=l1_l2(l1=0.1,l2=0.2)))
# model.add(Dense(16,activation='gelu',use_bias=True,kernel_regularizer=l1_l2(l1=0.01,l2=0.02)))
model.add(BatchNormalization())
model.add(Dense(32,activation=LeakyReLU(alpha=0.07),kernel_initializer='he_normal',use_bias=True,kernel_regularizer=l1_l2(l1=0.001,l2=0.01)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(64,activation='gelu',use_bias=True,kernel_regularizer=l1_l2(l1=0.001,l2=0.01)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(32,activation='gelu',use_bias=True,kernel_regularizer=l1_l2(l1=0.001,l2=0.01)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(16,activation='gelu',use_bias=True,kernel_regularizer=l1_l2(l1=0.001,l2=0.01)))
# model.add(Dense(1, activation="linear"))
model.add(Dense(2, activation="softmax"))
callback = EarlyStopping(monitor='val_loss', patience=2)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[ 'accuracy' ])

In [ ]:
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.1)

In [ ]:
batch_size=16
history = model.fit(X, y, validation_split=0.25,batch_size=batch_size,epochs=100, callbacks=[callback])

Epoch 1/100
157/157 [==============================] - 6s 10ms/step - loss: 2.4989 - accuracy: 0.8035 - val_loss: 2.0538 - val_accuracy: 0.8417
Epoch 2/100
157/157 [==============================] - 1s 8ms/step - loss: 1.6974 - accuracy: 0.8607 - val_loss: 1.4762 - val_accuracy: 0.8441
Epoch 3/100
157/157 [==============================] - 2s 12ms/step - loss: 1.3104 - accuracy: 0.8647 - val_loss: 1.1701 - val_accuracy: 0.8525
Epoch 4/100
157/157 [==============================] - 2s 11ms/step - loss: 1.0842 - accuracy: 0.8659 - val_loss: 0.9849 - val_accuracy: 0.8597
Epoch 5/100
157/157 [==============================] - 1s 8ms/step - loss: 0.9382 - accuracy: 0.8727 - val_loss: 0.8699 - val_accuracy: 0.8621
Epoch 6/100
157/157 [==============================] - 1s 8ms/step - loss: 0.8361 - accuracy: 0.8731 - val_loss: 0.7789 - val_accuracy: 0.8741
Epoch 7/100
157/157 [==============================] - 1s 8ms/step - loss: 0.7862 - accuracy: 0.8647 - val_loss: 0.7249 - val_accuracy: 0.8

In [ ]:
accuracy_score(y_test, np.around(model.predict(X_test)))

11/11 [==============================] - 0s 2ms/step


0.9011976047904192

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=batch_size)
print( 'Loss = {} and Accuracy = {} %'.format( results[0] , results[1] * 100 ) )

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
k = 10
kf = KFold(n_splits=k,shuffle=True,random_state=1234)
acc = np.empty(k)
for i,(idxtrain, idxtest) in enumerate(kf.split(x)):
  xtrain = x[idxtrain]
  xtest = x[idxtest]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  model = Sequential()
  model.add(BatchNormalization())
  model.add((Dense(32,activation=LeakyReLU(alpha=0.07),kernel_initializer='he_normal',use_bias=True,kernel_regularizer=l1_l2(l1=0.001,l2=0.01))))
  model.add(BatchNormalization())
  model.add(Dense(64,activation='gelu',use_bias=True,kernel_regularizer=l1_l2(l1=0.001,l2=0.01)))
  model.add(BatchNormalization())
  model.add(Dense(32,activation='gelu',use_bias=True,kernel_regularizer=l1_l2(l1=0.001,l2=0.01)))
  model.add(BatchNormalization())
  model.add(Dense(16,activation='gelu',use_bias=True,kernel_regularizer=l1_l2(l1=0.001,l2=0.01)))
  model.add(Dense(1, activation="linear")) # Here we collect and add the outputs produced by th eprevious layer
  # Compile model: The model is initialized with the Adam optimizer and then it is compiled.
  callback = EarlyStopping(monitor='metrics', patience=3)
  model.compile(optimizer=optimizer, loss='mean_squared_error')
  model.fit(xtrain,ytrain,validation_split=0.25,batch_size=25,epochs=100,verbose=0)
  acc[i] = accuracy_score(ytest,model.predict(xtest))
  print(acc[i])
print(acc.mean())